In [1]:
# auto reload libraries (you do need to re-import libraries if you make changes)
%load_ext autoreload
%autoreload 2

# base 
import pandas as pd
import numpy as np
from pprint import pprint
import os

# preprocessing 
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import SimpleImputer, IterativeImputer,KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import PolynomialFeatures
# variable importance
from sklearn.inspection import permutation_importance

# visualize
import matplotlib.pyplot as plt

# models
from sklearn.svm import SVR

# own defined functions/classes 
from preprocessing.preprocess_pipe import MakeLowerCase
from preprocessing.preprocess_pipe import HdResolutionCategorizer
from preprocessing.preprocess_pipe import StorageCategorizer
from preprocessing.preprocess_pipe import SsdCategorizer

from preprocessing.preprocess_pipe import print_missing
from preprocessing.preprocess_pipe import calculate_perf
from preprocessing.preprocess_pipe import custom_scoring_func_single_p

from modelling.weight_samples import weights_samples

from postprocessing.postprocessing import plot_predictions_results
from postprocessing.postprocessing import plot_residuals

In [2]:
# global parameters
pd.set_option('display.max_columns', 500)
# style for plotting
plt.style.use('ggplot')
# interactive plotting
%matplotlib widget
# run grid search
RUN_GRID_SEARCH = True
# set working directory
uppath = lambda _path, n: os.sep.join(_path.split(os.sep)[:-n])
__file__ = 'C:\\Users\\Pieter-Jan\\Documents\\KuLeuven\\Semester2\\AA\\AAForB_Assignment2\\code\\PieterJan'
out = uppath(__file__, 2)
os.chdir(out)
print(os.getcwd())

C:\Users\Pieter-Jan\Documents\KuLeuven\Semester2\AA\AAForB_Assignment2


# Step 1: import data and split in train and validation set
The validation set is more our own kind of test set

In [3]:
# read in trainig and validation data
# use the same data split as we did in R
df_all_train = pd.read_csv("data\\train.csv", sep=',')
df_test = pd.read_csv("data\\test.csv", sep=',')

print(f'Dimensions of all training data {df_all_train.shape}')
print(f'Dimension test data {df_test.shape}')

Dimensions of all training data (510, 22)
Dimension test data (222, 20)


In [4]:
# split in training and validation set
df_train = df_all_train.sample(frac=0.75, random_state=0, replace=False)
df_val = df_all_train.drop(df_train.index)

# reset index, if you don't resit missing rows get inserted in the pipeline
# see: https://stackoverflow.com/questions/31323499/sklearn-error-valueerror-input-contains-nan-infinity-or-a-value-too-large-for
df_train = df_train.reset_index().drop('index',axis=1)
df_val = df_val.reset_index().drop('index',axis=1)

# Step 2: define pipeline

In [5]:
#Numerical features to pass down the numerical pipeline 
numerical_features = ['screen_size' ,'pixels_x','pixels_y',
                      'ram', 'weight','ssd','storage']

#Categrical features to pass down the categorical pipeline 
categorical_features = ['brand','base_name', 'screen_surface','touchscreen',
                        'cpu','cpu_details','detachable_keyboard',
                        'discrete_gpu','gpu', 'os','os_details']

# define all unique features
features = np.unique(numerical_features).tolist() + np.unique(categorical_features).tolist()

# target variables
target = ['min_price','max_price']

In [6]:
# train
X_train = df_train[features]
y_train = df_train[target]

# validation (this is kind our own test set)
X_val  = df_val[features]
y_val = df_val[target]

# train_validation (this is all training data we have) for fitting the model
X_all_train = df_all_train[features]
y_all_train = df_all_train[target]

# test
X_test = df_test[features]

In [7]:
# you can add many more and 
# you can even define custom preprocessing steps like 'MakeLowerCase()'

# pipeline  uses only numerical features,
numeric_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    #('imputer', KNNImputer(n_neighbors=5, weights='uniform',metric='nan_euclidean')),
    ('imputer', IterativeImputer(missing_values=np.nan, random_state=0, n_nearest_features=None, sample_posterior=True)),
    #('transformation', PowerTransformer(method='yeo-johnson',standardize=False)),
     #Scale features using statistics that are robust to outliers.
    ('scaler', RobustScaler(with_centering=True, with_scaling=True, quantile_range=(10.0, 90.0)))]) 

# pipeline use only categorical features
categorical_transformer = Pipeline(steps=[ 
    ('lowercase', MakeLowerCase()), # lower cases all columns containing strings
    #('sd_category' ,SsdCategorizer(drop_original_feature=True)),
    #('storage_category', StorageCategorizer(drop_original_feature=True)),
    ('imputer', SimpleImputer(missing_values=np.nan, strategy="most_frequent")),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# add both preprocessing pipelines in one pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

In [8]:
# this is how the preporcessed pipeline looks like (just to have an idea)
pd.DataFrame(preprocessor.fit_transform(X_train)).shape

(382, 1343)

# Step 3: add models to pipeline

Possible to fit multiple target variabels, so you **don't** need to fit a different models for min. price and max. price

### A) Training and parameter tuning

##### 1) Automatic tuning via grid search

I will only **do the tuning for the minimum price** and use the found parameters also for the maximum price

In [9]:
# define model: I just add some default parameters but you could
# also just write: RandomForestRegressor() since we will perform a grid search 
# to find good hyperparameter values
model_min_p = SVR()
model_max_p = SVR()

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(model_min_p.get_params())

# add to pipeline
# 1) min price
pipeline_min_p = Pipeline(memory=None,
              steps=[('preprocessor', preprocessor),
                     ('regressor', model_min_p)])


# add transformation on the target variable, by default power transformation 
# also performs standardization after performing the power transformation
# and back transform to the original space when outputting predictions 

# 1) min price
transformer_target = PowerTransformer(method='yeo-johnson',standardize=False)
scale_target = RobustScaler(with_centering=True, with_scaling=True, quantile_range=(10.0, 90.0))
pipeline_y = Pipeline(memory=None,
              steps=[('transformer', transformer_target)])

pipeline_min_p_update = TransformedTargetRegressor(regressor=pipeline_min_p, 
                                         transformer=pipeline_y)

Parameters currently in use:

{'C': 1.0,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.1,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}


In [10]:
# kernelstring, optional (default=’rbf’)
# Specifies the kernel type to be used in the algorithm. It must be one of ‘linear’, 
# ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’ or a callable. 
# If none is given, ‘rbf’ will be used. If a callable is given it is used to precompute the kernel matrix.

kernel = ['linear', 'poly', 'rbf', 'sigmoid']

# degreeint, optional (default=3)
# Degree of the polynomial kernel function (‘poly’). Ignored by all other kernels.
degree = [1, 3, 5, 7, 9, 11, 13]

# tol, optional (default=1e-3)
# Tolerance for stopping criterion.
tol = [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3 ,0.5]

# gamma{‘scale’, ‘auto’} or float, optional (default=’scale’)
# Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.
#   - if gamma='scale' (default) is passed then it uses 1 / (n_features * X.var()) as value of gamma,
#   - if ‘auto’, uses 1 / n_features.

gamma = ['scale','auto']

# coef0 float, optional (default=0.0)
# Independent term in kernel function. It is only significant in ‘poly’ and ‘sigmoid’.

coef0 = [0, 0.1, 0.25, 1, 1.25, 1.5, 2, 2.5]

# C float, optional (default=1.0)
# Regularization parameter. The strength of the regularization is inversely 
# proportional to C. Must be strictly positive. The penalty is a squared l2 penalty
C = [0.001, 0.01, 0.1, 0.25, 0.5, 1, 2, 10]

       
# epsilon float, optional (default=0.1)
# Epsilon in the epsilon-SVR model. It specifies the epsilon-tube within which no penalty 
# is associated in the training loss function with points predicted within a distance epsilon from the actual value.
epsilon = [0.001, 0.05,0.025, 0.1, 0.2, 0.3]
      
# shrinking boolean, optional (default=True)
# Whether to use the shrinking heuristic.    
shrinking = [True, False]

# Create the random grid
random_grid = {
   'regressor__regressor__kernel': kernel,
   'regressor__regressor__degree': degree,
   'regressor__regressor__tol': tol,
   'regressor__regressor__gamma': gamma,
   'regressor__regressor__coef0': coef0,
   'regressor__regressor__C': C,
   'regressor__regressor__epsilon': epsilon,
   'regressor__regressor__shrinking': shrinking    
}

In [ ]:
# define random search (and narrow down time grid search)
if RUN_GRID_SEARCH:
    min_p_random_search = RandomizedSearchCV(
       estimator = pipeline_min_p_update, 
       param_distributions = random_grid, n_iter = 50,
       cv = 5, verbose=2, random_state=1, n_jobs = -1, refit=True,
       scoring=make_scorer(custom_scoring_func_single_p, greater_is_better=False)
    )

    # run grid search and refit with best hyper parameters
    weights_train_min_p =  weights_samples(df=y_train.iloc[:,0], order=0, plot_weights=False)
    min_p_random_search.fit(X_train, y_train.iloc[:,0])  
    print(min_p_random_search.best_params_)    
    print(min_p_random_search.best_score_)
    
# have look at the best hyperparameters and their respective performance (maybe also look at the sd)
pd.DataFrame(min_p_random_search.cv_results_).sort_values(
    by=['mean_test_score'],ascending=False).head(5)

Sum weights: 1.0
Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   40.0s


##### 2) Manual parameters tuning

In [13]:
# train your final model on all data with best parameters 

# 1) min price
model_min_p = SVR(
    tol = 0.0001,
    shrinking = True,
    kernel = 'rbf',
    gamma = 'scale',
    epsilon = 0.001,
    degree = 7,
    coef0 = 0.1,
    C = 1
)

# 2)  min price
model_max_p = SVR(   
    tol = 0.0001,
    shrinking = True,
    kernel = 'rbf',
    gamma = 'scale',
    epsilon = 0.001,
    degree = 7,
    coef0 = 0.1,
    C = 1
)


# add to pipeline

# 1) min price
pipeline_min_p = Pipeline(memory=None,
              steps=[('preprocessor', preprocessor),
                     ('regressor', model_min_p)],
              verbose=True)

# 2) min price
pipeline_max_p = Pipeline(memory=None,
              steps=[('preprocessor', preprocessor),
                     ('regressor', model_max_p)],
              verbose=True)

# again add transformer for target variable

# 1) min price
pipeline_min_p_update = TransformedTargetRegressor(regressor=pipeline_min_p, 
                                         transformer=pipeline_y)
# 2) max price
pipeline_max_p_update = TransformedTargetRegressor(regressor=pipeline_max_p, 
                                         transformer=pipeline_y)

# fit final model on all training data we have at hand

# 1) min price
weights_train_min_p = weights_samples(y_train.iloc[:,0], order=0)
pipeline_min_p_update.fit(X_train, y_train.iloc[:,0])

# 2) max price
weights_train_max_p = weights_samples(y_train.iloc[:,1], order=0)
pipeline_max_p_update.fit(X_train, y_train.iloc[:,1])

Sum weights: 1.0
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.1s
[Pipeline] ......... (step 2 of 2) Processing regressor, total=   0.2s
Sum weights: 1.0
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.1s
[Pipeline] ......... (step 2 of 2) Processing regressor, total=   0.3s


TransformedTargetRegressor(check_inverse=True, func=None, inverse_func=None,
                           regressor=Pipeline(memory=None,
                                              steps=[('preprocessor',
                                                      ColumnTransformer(n_jobs=None,
                                                                        remainder='drop',
                                                                        sparse_threshold=0.3,
                                                                        transformer_weights=None,
                                                                        transformers=[('num',
                                                                                       Pipeline(memory=None,
                                                                                                steps=[('imputer',
                                                                                                        Itera

In [14]:
# performance on data where the model was fit one (should be very low)
pred_train_min_p = pipeline_min_p_update.predict(X_train)
pred_train_max_p = pipeline_max_p_update.predict(X_train)

# calculate performance
pred_train = pd.DataFrame([pred_train_min_p,pred_train_max_p]).T
calculate_perf(y_train, pred_train)

{'minimum price': 418.9103259317542,
 'maximum price': 435.94005039542657,
 'total error': 854.8503763271808}

##### Performance on validation data

In [15]:
# performance on validation data
pred_val_min_p = pipeline_min_p_update.predict(X_val)
pred_val_max_p = pipeline_max_p_update.predict(X_val)

# calculate performance 
pred_val = pd.DataFrame([pred_val_min_p,pred_val_max_p]).T
calculate_perf(y_val, pred_val)

{'minimum price': 346.16028764762916,
 'maximum price': 362.34067023146133,
 'total error': 708.5009578790905}

In [16]:
# save predictions on validation data
# submission format
submission_format_validation = pd.DataFrame.from_dict(
 {'ID':df_val['id'].values,
 'MIN':pred_val_min_p,
 'MAX':pred_val_max_p}).set_index('ID')

# write to csv
submission_format_validation.to_csv('code\\PieterJan\\python\\output\\validation\\suppor_vector_machine.csv' ,
                            header=True, index=True, sep=',')

### C) Post processing


In [17]:
fig, axs = plt.subplots(2, 2, figsize=(9, 7))
axs = np.ravel(axs)

# fitted against true predictions minimum price
plot_predictions_results(ax=axs[0], 
                        y_true=y_val.iloc[:,0], 
                        y_pred=pred_val_min_p, 
                        title="SVM Min. Price Test Set", 
                        log_scale=True)

# fitted against true predictions maximum price
plot_predictions_results(ax=axs[1], 
                        y_true=y_val.iloc[:,1], 
                        y_pred=pred_val_max_p, 
                        title="SVM Max. Price Test Set", 
                        log_scale=True)

# residuals minimum price
plot_residuals(ax=axs[2], 
               y_true=y_val.iloc[:,0], 
               y_pred=pred_val_min_p, 
               title="", 
               log_scale=False,
               order=1)

# residuals maximum price
plot_residuals(ax=axs[3], 
               y_true=y_val.iloc[:,1], 
               y_pred=pred_val_max_p, 
               title="", 
               log_scale=False,
               order=1)
fig.tight_layout()
plt.savefig('code\\PieterJan\\python\\output\\figures\\support_vector_machines\\fig1.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
plt.figure(figsize=(8,4))
plt.plot(pred_val_max_p - pred_val_min_p, label="Predictions Test", linestyle="--")
plt.plot(y_val.iloc[:,1] - y_val.iloc[:,0], label="Truth Test", linestyle=':')
plt.xlabel("Observations")
plt.ylabel("Max. Price - Min. Price")
plt.legend(loc="upper right");
plt.savefig('code\\PieterJan\\python\\output\\figures\\support_vector_machines\\fig2.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
vip_val_min_p = permutation_importance(pipeline_min_p_update, X_val, y_val.iloc[:,0], n_repeats=10,
                                random_state=1, n_jobs=3)

vip_val_max_p = permutation_importance(pipeline_min_p_update, X_val, y_val.iloc[:,1], n_repeats=10,
                                random_state=1, n_jobs=3)

sorted_idx_min_p = vip_val_min_p.importances_mean.argsort()
sorted_idx_max_p = vip_val_max_p.importances_mean.argsort()

In [20]:
fig, ax = plt.subplots(1, 2, figsize=(9, 4))
ax = np.ravel(ax)
# minimum price
ax[0].boxplot(vip_val_min_p.importances[sorted_idx_min_p].T,
           vert=False, labels=X_val.columns[sorted_idx_min_p])
ax[0].set_title("Permutation Importances: Min. Price (Test set)", fontsize=9)
ax[0].xaxis.set_tick_params(labelsize=8)
ax[0].yaxis.set_tick_params(labelsize=8)

# maximum price
ax[1].boxplot(vip_val_min_p.importances[sorted_idx_max_p].T,
           vert=False, labels=X_val.columns[sorted_idx_max_p])
ax[1].set_title("Permutation Importances: Max. Price (Test set)", fontsize=9)
ax[1].xaxis.set_tick_params(labelsize=8)
ax[1].yaxis.set_tick_params(labelsize=8)
fig.tight_layout()
plt.show()
plt.savefig('code\\PieterJan\\python\\output\\figures\\support_vector_machines\\fig3.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### D) Predictions test data

In [18]:
# train your final model on all data with best parameters 

# 1) min price
model_min_p_final = SVR(
    tol = 0.0001,
    shrinking = False,
    kernel = 'rbf',
    gamma = 'scale',
    epsilon = 0.001,
    degree = 5,
    coef0 = 0.25,
    C = 1
)

# 2)  min price
model_max_p_final = SVR(
    tol = 0.0001,
    shrinking = False,
    kernel = 'rbf',
    gamma = 'scale',
    epsilon = 0.001,
    degree = 5,
    coef0 = 0.25,
    C = 1
)


# add to pipeline

# 1) min price
pipeline_min_p_final = Pipeline(memory=None,
              steps=[('preprocessor', preprocessor),
                     ('regressor', model_min_p_final)],
              verbose=True)

# 2) min price
pipeline_max_p_final = Pipeline(memory=None,
              steps=[('preprocessor', preprocessor),
                     ('regressor', model_max_p_final)],
              verbose=True)

# again add transformer for target variable

# 1) min price
pipeline_min_p_final = TransformedTargetRegressor(regressor=pipeline_min_p_final, 
                                         transformer=pipeline_y)
# 2) max price
pipeline_max_p_final = TransformedTargetRegressor(regressor=pipeline_max_p_final, 
                                         transformer=pipeline_y)

# fit final model on all training data we have at hand

# 1) min price
weights_all_train_min_p = weights_samples(y_all_train.iloc[:,0], order=2)
pipeline_min_p_final.fit(X_train, y_train.iloc[:,0])

# 2) max price
weights_all_train_max_p = weights_samples(y_train.iloc[:,1], order=2)
pipeline_max_p_final.fit(X_train, y_train.iloc[:,1])

Sum weights: 1.0
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.2s
[Pipeline] ......... (step 2 of 2) Processing regressor, total=   0.0s
Sum weights: 1.0
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.3s
[Pipeline] ......... (step 2 of 2) Processing regressor, total=   0.0s


In [19]:
# performance on all data where the model was fit one (should be very low)
pred_all_train_min_p = pipeline_min_p_final.predict(X_all_train)
pred_all_train_max_p = pipeline_max_p_final.predict(X_all_train)

# calculate performance 
pred_all_train = pd.DataFrame([pred_all_train_min_p, pred_all_train_max_p]).T
calculate_perf(y_all_train, pred_all_train)

{'minimum price': 62.55169329839375,
 'maximum price': 63.131304433276924,
 'total error': 125.68299773167067}

In [20]:
# make predictions on test data
pred_test_min_p = pipeline_min_p_final.predict(X_test)
pred_test_max_p = pipeline_max_p_final.predict(X_test)

# submission format
submission_format_test = pd.DataFrame.from_dict(
 {'ID':df_test['id'].values,
 'MIN':pred_test_min_p,
 'MAX':pred_test_max_p}).set_index('ID')

# write to csv
submission_format_test.to_csv('code\\PieterJan\\python\\output\\submission\\suppor_vector_machine.csv' ,
                            header=True, index=True, sep=',')